In [1]:
# 构建DSSM模型
# 模型构建部分主要是将输入的user特征以及item特征处理完后送入两侧的DNN结构
def DSSM(user_feature_columns, item_feature_columns, dnn_units=[64, 32], 
        temp=10, task='binary'):
    # 构建所有特征的Input层和Embedding层
    feature_encode = FeatureEncoder(user_feature_columns + item_feature_columns)
    feature_input_layers_list = list(feature_encode.feature_input_layer_dict.values())

    # 特征处理
    user_dnn_input, item_dnn_input = process_feature(user_feature_columns,\
        item_feature_columns, feature_encode)

    # 构建模型的核心层
    if len(user_dnn_input) >= 2:
        user_dnn_input = Concatenate(axis=1)(user_dnn_input)
    else:
        user_dnn_input = user_dnn_input[0]
    if len(item_dnn_input) >= 2:
        item_dnn_input = Concatenate(axis=1)(item_dnn_input)
    else:
        item_dnn_input = item_dnn_input[0]
    user_dnn_input = Flatten()(user_dnn_input) 
    item_dnn_input = Flatten()(item_dnn_input)
    user_dnn_out = DNN(dnn_units)(user_dnn_input)
    item_dnn_out = DNN(dnn_units)(item_dnn_input)


    # 计算相似度
    scores = CosinSimilarity(temp)([user_dnn_out, item_dnn_out]) # (B,1)
    # 确定拟合目标
    output = PredictLayer()(scores)
    # 根据输入输出构建模型
    model = Model(feature_input_layers_list, output)
    return model 


In [ ]:
# CosinSimilarity相似度计算
def call(self, inputs, **kwargs):
    """inputs 是一个列表"""
    query, candidate = inputs 
    # 计算两个向量的二范数
    query_norm = tf.norm(query, axis=self.axis) # (B, 1)
    candidate_norm = tf.norm(candidate, axis=self.axis)
    # 计算向量点击，即內积操作
    scores = tf.reduce_sum(tf.multiply(query, candidate), axis=-1)#(B,1)
    # 相似度除以二范数, 防止除零
    scores = tf.divide(scores, query_norm * candidate_norm + 1e-8)
    # 对score的范围限制到(-1, 1)之间
    scores = tf.clip_by_value(scores, -1, 1)
    # 乘以温度系数
    score = scores * self.temperature 
    return score 
